# Zenhub & Github Issue Import and Formatting

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#functions">Functions</a></li>
<li><a href="#exploration">Data Exploration</a></li>
<li><a href="#csv">Writing to CSV</a></li>
</ul>

<a id='intro'></a>
## Introduction

Before process starts, first import the necessary libraries and initialise global parameters.

* Project parameters could be find under project settings.
* For Github token, go to `Settings \ Developer settings \ Personal access tokens` to generate a new token.
* For Zenhub token, go to https://app.zenhub.com/dashboard/tokens and generate a new token.

In [11]:
import requests
import pandas as pd
import numpy as np
import io
import json
import time
import datetime

########## SET BELOW PARAMETERS ##########
# Project Parameters
OWNER = ""
REPOSITORY_ID = ""
REPOSITORY_NAME = ""

# API TOKENS
ZENHUB_API_TOKEN = ""
GITHUB_API_TOKEN = ""

# URLS
ZENHUB_REPO_URL = "https://.../p1/repositories/" # url ends with "/repositories/"
GITHUB_REPO_URL = "https://.../api/v3/repos/" # url ends with "/repos/"
########## SET ABOVE PARAMETERS ##########


# Zenhub URL Parameters
ZENHUB_URL_EPICS = (ZENHUB_REPO_URL + REPOSITORY_ID + "/epics")
ZENHUB_URL_ISSUES = (ZENHUB_REPO_URL + REPOSITORY_ID + "/issues/")
ZENHUB_HEADERS = {"X-Authentication-Token": ZENHUB_API_TOKEN}

# Github URL Parameters
GITHUB_ISSUES_URL = (GITHUB_REPO_URL + OWNER + "/" + REPOSITORY_NAME + "/issues")
GITHUB_HEADERS = {"Authorization": ("Bearer " + GITHUB_API_TOKEN)}

# APIs Rate Limit waiting time
SLEEP_TIME = 60

<a id='functions'></a>
## Functions

In this section, functions will be defined

* get_epic_issues_json
* get_issue_events_json
* get_issue_labels_and_type
* get_all_issues
* get_event_df
* calculate_time_spent_per_column
* insert_first_date_per_column

In [2]:
# a function returning all issues under an epic in json format
def get_epic_issues_json(epic):

    # edit URL to get a single epic
    temp_issues_url = ZENHUB_URL_EPICS + "/" + epic

    # call the API endpoint and sleep 60sec if call returns API Limit exception
    issues_response = requests.get(temp_issues_url, headers=ZENHUB_HEADERS)
    if issues_response.status_code == 403:
        time.sleep(SLEEP_TIME)
        issues_response = requests.get(temp_issues_url, headers=ZENHUB_HEADERS)

    return issues_response.json()

In [3]:
# a function returning all events happened in the lifetime of an issue in json format
def get_issue_events_json(issue_id):

    # edit URL to get issues events
    temp_events_url = ZENHUB_URL_ISSUES + issue_id + "/events"

    # call the API endpoint and sleep 60sec if call returns API Limit exception
    events_response = requests.get(temp_events_url, headers=ZENHUB_HEADERS)
    if events_response.status_code == 403:
        time.sleep(SLEEP_TIME)
        events_response = requests.get(temp_events_url, headers=ZENHUB_HEADERS)

    return events_response.json()

In [4]:
# a function returning label set and type by parsing a json input
def get_issue_labels_and_type(labels_json):

    # parse the json and get the labels list
    labels = ""
    for label in labels_json:
        if labels == "":
            labels = label["name"]
        else:
            labels = labels + "," + label["name"]

    # find the type of the story by looking at the label
    issue_type = ""
    if labels.find("story") != -1:
        issue_type = "story"
    elif labels.find("bug") != -1:
        issue_type = "bug"
    elif labels.find("discovery") != -1:
        issue_type = "discovery"
    elif labels.find("spike") != -1:
        issue_type = "spike"
    elif labels.find("setup") != -1:
        issue_type = "setup"
    elif labels.find("security") != -1:
        issue_type = "security"
    elif labels.find("tech-debt") != -1:
        issue_type = "techdebt"
    elif labels.find("documentation") != -1:
        issue_type = "documentation"

    return labels, issue_type

In [5]:
# a function returning all issues under your Github repository in json format
def get_all_issues(page_number):

    # call the API endpoint and sleep 60sec if call returns API Limit exception
    issues_response = requests.get(
        GITHUB_ISSUES_URL, headers=GITHUB_HEADERS, params={"page": str(page_number)}
    )
    if issues_response.status_code == 403:
        time.sleep(SLEEP_TIME)
        issues_response = requests.get(
            GITHUB_ISSUES_URL, headers=GITHUB_HEADERS, params={"page": str(page_number)}
        )

    return issues_response.json()

In [6]:
# a function parsing json and returning issue events in dataframe format
def get_event_df(events_json):

    # initialise the dataframe first
    df_events_col_names = ["from_pipeline", "to_pipeline", "created_at"]
    df_events = pd.DataFrame(columns=df_events_col_names)

    # parse json and append data to dataframe
    for event in events_json:

        # for kanban board issue transfer events, we will use the events with "transferIssue" type
        if event["type"] == "transferIssue":
            from_pipeline = event["from_pipeline"]["name"]
            to_pipeline = event["to_pipeline"]["name"]
            created_at = datetime.datetime.strptime(
                event["created_at"], "%Y-%m-%dT%H:%M:%S.%fZ"
            ).date()
            df_events = df_events.append(
                {
                    "from_pipeline": from_pipeline,
                    "to_pipeline": to_pipeline,
                    "created_at": created_at,
                },
                ignore_index=True,
            )

    return df_events

In [7]:
# a function calculating times spent on each column of your kanban board (except 'Done' column)
def calculate_time_spent_per_column(df_events, created_at):
    
    # initialise parameters
    new_issues = tech_debt = spike = in_analysis = ready_to_pick = in_progress = sign_off = done = 0
    prev_date = created_at
    
    # iterate on events dataframe
    df_events = df_events.sort_values(by=['created_at'])
    for row in df_events.iterrows():
        
        from_pipeline = row[1]['from_pipeline']
        action_date = row[1]['created_at']
        delta = (action_date - prev_date).days # 2 moves happened at the same day, delta is 0!!!
        prev_date = row[1]['created_at']
    
        # calculate "how many days a ticket stand in each column"
        if (from_pipeline == "New Issues"):
            new_issues = new_issues + delta
        elif (from_pipeline == "Tech Debt"):
            tech_debt = tech_debt + delta
        elif (from_pipeline.startswith("Spike")):
            spike = spike + delta
        elif (from_pipeline == "In analysis"):
            in_analysis = in_analysis + delta
        elif (from_pipeline == "Ready to pick"):
            ready_to_pick = ready_to_pick + delta
        elif (from_pipeline == "In Progress"):
            in_progress = in_progress + delta
        elif (from_pipeline == "Sign off"):
            sign_off = sign_off + delta
    
    return new_issues, tech_debt, spike, in_analysis, ready_to_pick, in_progress, sign_off


In [8]:
# a function finding first date when an issue moved to a column on your kanban board (except 'New Issues' column)
# NOTE: an issue can be moved to a column multiple times, this function will find the date for only the first times
def insert_first_date_per_column(df_events, df_issues):

    # iterate on events dataframe
    df_events = df_events.sort_values(by=["created_at"], ascending=False)
    for row in df_events.iterrows():
        to_pipeline = row[1]["to_pipeline"]
        action_date = row[1]["created_at"]

        # find "ticket's first date in each column"
        if to_pipeline == "Tech Debt":
            df_issues.loc[df_issues.issue_id == issue_id, "tech_debt_fd"] = action_date
        elif to_pipeline.startswith("Spike"):
            df_issues.loc[df_issues.issue_id == issue_id, "spike_fd"] = action_date
        elif to_pipeline == "In analysis":
            df_issues.loc[
                df_issues.issue_id == issue_id, "in_analysis_fd"
            ] = action_date
        elif to_pipeline == "Ready to pick":
            df_issues.loc[
                df_issues.issue_id == issue_id, "ready_to_pick_fd"
            ] = action_date
        elif to_pipeline == "In Progress":
            df_issues.loc[
                df_issues.issue_id == issue_id, "in_progress_fd"
            ] = action_date
        elif to_pipeline == "Sign off":
            df_issues.loc[df_issues.issue_id == issue_id, "sign_off_fd"] = action_date
        elif to_pipeline == "Done":
            df_issues.loc[df_issues.issue_id == issue_id, "done_fd"] = action_date

    return

<a id='exploration'></a>
## Data Exploration

Time for the real job: issues dataframe initialisation, and filling with data.

In [9]:
# initialise the issues dataframe
col_names = [
    "epic_id",
    "issue_id",
    "title",
    "labels",
    "issue_type",
    "state",
    "created_at",
    "events",
    "new_issues",
    "tech_debt",
    "spike",
    "in_analysis",
    "ready_to_pick",
    "in_progress",
    "sign_off",
    "tech_debt_fd",
    "spike_fd",
    "in_analysis_fd",
    "ready_to_pick_fd",
    "in_progress_fd",
    "sign_off_fd",
    "done_fd",
]
df_issues = pd.DataFrame([], columns=col_names)

In [10]:
# NOTE: THIS CELL'S EXECUTION CAN TAKE A WHILE (minutes) DUE TO API LIMIT SLEEPING!!!!

# all functions are defined and initialisations made
# now we will get all issues on our github repo by pagination
page_number = 1
issues_json = get_all_issues(page_number)

# continue since next page of API response is empty!
while len(issues_json) > 0:

    ## loop on issues at a response
    for issue in issues_json:

        # get issue parameters
        issue_id = str(issue["number"])
        created_at = datetime.datetime.strptime(
            issue["created_at"], "%Y-%m-%dT%H:%M:%SZ"
        ).date()
        state = issue["state"]
        title = issue["title"]
        labels, issue_type = get_issue_labels_and_type(issue["labels"])

        # get events json and events data frame of issue
        events_json = get_issue_events_json(issue_id)
        df_events = get_event_df(events_json)

        # get delta calculations
        (
            new_issues,
            tech_debt,
            spike,
            in_analysis,
            ready_to_pick,
            in_progress,
            sign_off,
        ) = calculate_time_spent_per_column(df_events, created_at)

        # insert issue into issues data frame
        df_issues = df_issues.append(
            {
                "epic_id": "",
                "issue_id": issue_id,
                "title": title,
                "labels": labels,
                "issue_type": issue_type,
                "state": state,
                "created_at": created_at,
                "events": df_events.to_json(date_format="iso"),
                "new_issues": new_issues,
                "tech_debt": tech_debt,
                "spike": spike,
                "in_analysis": in_analysis,
                "ready_to_pick": ready_to_pick,
                "in_progress": in_progress,
                "sign_off": sign_off,
            },
            ignore_index=True,
        )

        # add "first date" when a ticket moved to a column
        insert_first_date_per_column(df_events, df_issues)

    # get next page's json file
    page_number = page_number + 1
    issues_json = get_all_issues(page_number)

print("over!")

over!


In [12]:
# NOTE: THIS CELL'S EXECUTION CAN TAKE A WHILE (minutes) DUE TO API LIMIT SLEEPING!!!!

# now we will get all epics on our github repo
epic_list = []
json_content = requests.get(ZENHUB_URL_EPICS, headers=ZENHUB_HEADERS).json()
for epic in json_content["epic_issues"]:
    epic_id = str(epic["issue_number"])

    # get all issues assigned to that epic, and update epic id of issue at data frame
    issues_json = get_epic_issues_json(epic_id)
    for issue in issues_json["issues"]:
        issue_id = str(issue["issue_number"])
        df_issues.loc[df_issues.issue_id == issue_id, "epic_id"] = epic_id

print("over!")

over!


In [13]:
df_issues.head(3)

,epic_id,issue_id,title,labels,issue_type,state,created_at,events,new_issues,tech_debt,...,ready_to_pick,in_progress,sign_off,tech_debt_fd,spike_fd,in_analysis_fd,ready_to_pick_fd,in_progress_fd,sign_off_fd,done_fd
0,225,439,Retire /filter-auto-populate-values,story 🍰,story,open,2020-03-26,"{""from_pipeline"":{""0"":""New Issues""},""to_pipeli...",0,0,...,0,0,0,NaN,NaN,2020-03-26,NaN,NaN,NaN,NaN
1,406,438,Reduce nodes to 2 for Redshift TEST cluster,"setup ⚙️,tech-debt 💸",setup,open,2020-03-26,"{""from_pipeline"":{""0"":""Sign off"",""1"":""Ready to...",0,0,...,0,0,0,NaN,NaN,NaN,2020-03-26,NaN,2020-03-26,2020-03-26
2,71,435,Update d_merchant_extended ingestion date to y...,"story 🍰,tech-debt 💸",story,open,2020-03-24,"{""from_pipeline"":{""0"":""In analysis"",""1"":""New I...",0,0,...,0,0,0,NaN,NaN,2020-03-24,2020-03-26,NaN,NaN,NaN


In [14]:
df_issues.describe()

,epic_id,issue_id,title,labels,issue_type,state,created_at,events,new_issues,tech_debt,...,ready_to_pick,in_progress,sign_off,tech_debt_fd,spike_fd,in_analysis_fd,ready_to_pick_fd,in_progress_fd,sign_off_fd,done_fd
count,354,354,354,354,354,354,354,354,354,354,...,354,354,354,30,22,176,198,247,179,268
unique,23,352,352,33,9,1,86,320,28,13,...,31,28,19,21,14,51,66,105,90,90
top,,92,SKU Insights API - data materialization & inge...,story 🍰,story,open,2019-09-18,"{""from_pipeline"":{""0"":""New Issues""},""to_pipeli...",0,0,...,0,0,0,2020-03-10,2019-10-18,2020-02-26,2020-02-03,2019-09-18,2020-03-25,2020-02-17
freq,99,2,2,127,184,354,16,5,284,340,...,206,164,187,3,6,17,12,9,6,18


<a id='csv'></a>
## Writing to CSV

It is time to write issues data into a CSV for external uses.

In [15]:
df_issues.to_csv("github_issues.csv")